# Berlin Housing Market Analysis - Hauptanalyse
## Datenbereinigung, Zusammenführung und Explorative Analyse

### Projektübersicht
Dieses Notebook führt die Hauptanalyse des Berliner Wohnungsmarktes durch. Es baut auf dem PLZ-Mapping aus `01_Data_Preprocessing.ipynb` auf und erstellt eine umfassende Analyse der drei Datensätze.

### Voraussetzungen
- `01_Data_Preprocessing.ipynb` wurde erfolgreich ausgeführt
- PLZ-Mapping-Tabelle liegt in `data/processed/berlin_plz_mapping.csv` vor
- Originaldaten befinden sich in `data/raw/`

### Analyseziele
1. **Datenbereinigung und -normalisierung** aller drei Datensätze
2. **Zusammenführung** in ein einheitliches Dataset
3. **Zeitreihenanalyse** der Mietpreisentwicklung
4. **Bezirksvergleiche** mit statistischen Tests
5. **Explorative Datenanalyse** mit umfassenden Visualisierungen

---

**Autor**: Berlin Housing Market Analysis Team  
**Datum**: 4. Juli 2025  
**Version**: 1.0

## 1. Import Required Libraries
Importieren der erforderlichen Python-Bibliotheken für Datenverarbeitung, Analyse und Visualisierung.

In [4]:
# Datenmanipulation und -analyse
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualisierung
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Statistik und Machine Learning
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Textverarbeitung und Regex
import re
from collections import Counter

# Datum und Zeit
from datetime import datetime

# Konfiguration
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 50)
plt.style.use('seaborn-v0_8')

print("Alle Bibliotheken erfolgreich importiert!")
print(f"Pandas Version: {pd.__version__}")
print(f"NumPy Version: {np.__version__}")
print(f"Matplotlib Version: {matplotlib.__version__}")
print(f"Seaborn Version: {sns.__version__}")
print(f"Verarbeitung gestartet am: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Alle Bibliotheken erfolgreich importiert!
Pandas Version: 2.3.0
NumPy Version: 2.3.0
Matplotlib Version: 3.10.3
Seaborn Version: 0.13.2
Verarbeitung gestartet am: 2025-07-04 04:21:25


## 2. Laden der bereinigten Daten
### 2.1 Bereinigte Datensätze laden
Laden der im Preprocessing-Notebook bereits bereinigten und normalisierten Datensätze.

In [6]:
# Lade die bereinigten Datensätze aus dem Preprocessing
print("=" * 60)
print("LADEN DER BEREINIGTEN DATENSÄTZE")
print("=" * 60)

# Lade das kombinierte und bereinigte Dataset
print("\nKombiniertes bereinigtes Dataset:")
combined_df = pd.read_csv('data/processed/berlin_housing_combined_clean.csv')
print(f"Zeilen: {combined_df.shape[0]:,}")
print(f"Spalten: {combined_df.shape[1]}")
print(f"Spalten: {list(combined_df.columns)}")

# Lade auch die einzelnen bereinigten Datensätze für detaillierte Analyse
print("\nEinzelne bereinigte Datensätze:")

# Dataset 2025 bereinigt
df_2025 = pd.read_csv('data/processed/dataset_2025_clean.csv')
print(f"Dataset 2025 bereinigt: {df_2025.shape[0]:,} Zeilen, {df_2025.shape[1]} Spalten")

# Dataset 2018-2019 bereinigt
df_2018_2019 = pd.read_csv('data/processed/dataset_2018_2019_clean.csv')
print(f"Dataset 2018-2019 bereinigt: {df_2018_2019.shape[0]:,} Zeilen, {df_2018_2019.shape[1]} Spalten")

# Dataset 2022 bereinigt
df_2022 = pd.read_csv('data/processed/dataset_2022_clean.csv')
print(f"Dataset 2022 bereinigt: {df_2022.shape[0]:,} Zeilen, {df_2022.shape[1]} Spalten")

# PLZ-Mapping als Referenz
plz_mapping_df = pd.read_csv('data/processed/berlin_plz_mapping.csv')
print(f"PLZ-Mapping: {plz_mapping_df.shape[0]:,} Einträge")

print(f"\nGesamtdatenpunkte (bereinigt): {combined_df.shape[0]:,}")
print(f"Verfügbare Bezirke: {combined_df['Bezirk'].nunique()}")
print(f"Zeitraum: {combined_df['Jahr'].min()} - {combined_df['Jahr'].max()}")

LADEN DER BEREINIGTEN DATENSÄTZE

Kombiniertes bereinigtes Dataset:
Zeilen: 19,779
Spalten: 96
Spalten: ['regio3', 'street', 'livingSpace', 'baseRent', 'totalRent', 'noRooms', 'floor', 'typeOfFlat', 'yearConstructed', 'Jahr', 'Dataset', 'ID', 'SORTE', 'PLZ_x', 'KALTMIETE', 'WARMMIETE', 'NEBENKOSTEN', 'KAUTION', 'HEIZUNGSKOSTEN', 'ZIMMER', 'PARKPLAETZE', 'WOHNFLAECHE', 'BAUJAHR', 'ZUSTAND', 'VERfÜGBAR AB', 'ENERGIEEFFIZIENSKLASSE', 'ENERGIEBEDARF/kWh/(m²*a)', 'ENERGIEASUWEIS', 'Etagenheizung', 'Zentralheizung', 'Ofenheizung', 'offener Kamin', 'Luft-/Wasser-Wärmepumpe', 'Gas', 'Öl', 'Solar', 'Strom', 'Holz', 'Fernwärme', 'Erdwärme', 'Pellets', 'Kohle', 'Flüssiggas', 'KfW 55', 'Niedrigenergie', 'KfW 40', 'KfW 60', 'KfW 70', 'Neubaustandard', 'möbliert', 'teilweise möbliert', 'Alarmanlage', 'Garage', 'Carport', 'Tiefgarage', 'Duplex', 'Stellplatz', 'Klimaanlage', 'Sauna', 'Schwimmbad', 'See', 'Berge', 'Personenaufzug', 'Lastenaufzug', 'Keller', 'Waschraum', 'Bibliothek', 'Terrasse', 'Balko

TypeError: '<=' not supported between instances of 'str' and 'int'

### 2.2 Datenstruktur und Qualität der bereinigten Daten
Analysiere die Struktur und Qualität der bereinigten Datensätze.

In [ ]:
# Detaillierte Analyse der bereinigten Datensätze
print("=" * 80)
print("DATENQUALITÄTSANALYSE - BEREINIGTE DATENSÄTZE")
print("=" * 80)

# Analyse des kombinierten Datasets
print(f"\n{'='*25} KOMBINIERTES DATASET {'='*25}")
print(f"Shape: {combined_df.shape}")
print(f"Memory usage: {combined_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Fehlende Werte prüfen
missing_info = combined_df.isnull().sum()
missing_pct = (missing_info / len(combined_df) * 100).round(2)

if missing_info.sum() > 0:
    print(f"\nFehlende Werte:")
    for col in missing_info[missing_info > 0].index:
        print(f"  {col}: {missing_info[col]} ({missing_pct[col]}%)")
else:
    print("\nKeine fehlenden Werte gefunden ✓")

# Datentypen
print(f"\nDatentypen:")
print(combined_df.dtypes.value_counts())

# Einzigartige Werte in Schlüsselspalten
print(f"\nSchlüsselspalten:")
print(f"  Bezirke: {combined_df['Bezirk'].nunique()} - {sorted(combined_df['Bezirk'].unique())}")
print(f"  Jahre: {combined_df['Year'].nunique()} - {sorted(combined_df['Year'].unique())}")
print(f"  Datenquellen: {combined_df['Datenquelle'].nunique()} - {sorted(combined_df['Datenquelle'].unique())}")

# Numerische Spalten Statistik
print(f"\nNumerische Spalten Statistik:")
numeric_cols = combined_df.select_dtypes(include=[np.number]).columns
if len(numeric_cols) > 0:
    print(combined_df[numeric_cols].describe())

# Analyse der einzelnen bereinigten Datensätze
datasets_clean = {
    '2025': df_2025,
    '2018_2019': df_2018_2019, 
    '2022': df_2022
}

for year, df in datasets_clean.items():
    print(f"\n{'='*20} DATASET {year} (BEREINIGT) {'='*20}")
    print(f"Shape: {df.shape}")
    print(f"Spalten: {list(df.columns)}")
    
    # Fehlende Werte
    missing = df.isnull().sum()
    if missing.sum() > 0:
        print(f"Fehlende Werte: {missing[missing > 0].to_dict()}")
    else:
        print("Keine fehlenden Werte ✓")
    
    # Preisstatistik
    if 'Price' in df.columns:
        print(f"Preis-Statistik:")
        print(f"  Min: {df['Price'].min():.2f} €")
        print(f"  Max: {df['Price'].max():.2f} €")
        print(f"  Median: {df['Price'].median():.2f} €")
        print(f"  Mean: {df['Price'].mean():.2f} €")

print(f"\n{'='*80}")
print("ZUSAMMENFASSUNG DER BEREINIGTEN DATEN")
print(f"{'='*80}")
print(f"Kombiniertes Dataset: {combined_df.shape[0]:,} Zeilen, {combined_df.shape[1]} Spalten")
print(f"Zeitraum: {combined_df['Year'].min()} - {combined_df['Year'].max()}")
print(f"Bezirke: {combined_df['Bezirk'].nunique()}")
print(f"Datenquellen: {combined_df['Datenquelle'].nunique()}")
print(f"Preisspanne: {combined_df['Price'].min():.0f} - {combined_df['Price'].max():.0f} €")

## 3. Explorative Datenanalyse (EDA)
### 3.1 Zeitliche Entwicklung der Mietpreise

Analysiere die Entwicklung der Mietpreise über die verfügbaren Zeiträume hinweg.

In [ ]:
# Zeitliche Entwicklung der Mietpreise analysieren
print("=" * 60)
print("ZEITLICHE ENTWICKLUNG DER MIETPREISE")
print("=" * 60)

# Grundlegende Zeitreihenstatistik
print("\nPreisstatistik nach Jahren:")
yearly_stats = combined_df.groupby('Jahr')['price'].agg(['count', 'mean', 'median', 'std', 'min', 'max']).round(2)
print(yearly_stats)

# Durchschnittspreis pro Jahr und Bezirk
print("\nDurchschnittspreis pro Jahr und Bezirk (Top 5 Bezirke):")
yearly_district_prices = combined_df.groupby(['Jahr', 'Bezirk'])['price'].mean().round(2)
top_districts = combined_df.groupby('Bezirk')['price'].mean().nlargest(5).index

for district in top_districts:
    district_data = yearly_district_prices[yearly_district_prices.index.get_level_values('Bezirk') == district]
    print(f"\n{district}:")
    for year, price in district_data.items():
        print(f"  {year[0]}: {price:.2f} €")

# Visualisierung der Preisentwicklung
plt.figure(figsize=(15, 10))

# Subplot 1: Durchschnittspreis pro Jahr
plt.subplot(2, 2, 1)
yearly_avg = combined_df.groupby('Jahr')['price'].mean()
plt.plot(yearly_avg.index, yearly_avg.values, marker='o', linewidth=2, markersize=8)
plt.title('Durchschnittliche Mietpreise über die Jahre', fontsize=14, fontweight='bold')
plt.xlabel('Jahr')
plt.ylabel('Durchschnittspreis (€)')
plt.grid(True, alpha=0.3)
for year, price in yearly_avg.items():
    plt.annotate(f'{price:.0f}€', (year, price), textcoords="offset points", xytext=(0,10), ha='center')

# Subplot 2: Boxplot der Preisverteilung pro Jahr
plt.subplot(2, 2, 2)
combined_df.boxplot(column='price', by='Jahr', ax=plt.gca())
plt.title('Preisverteilung pro Jahr')
plt.xlabel('Jahr')
plt.ylabel('Preis (€)')
plt.suptitle('')  # Entfernt den automatischen Titel

# Subplot 3: Anzahl Einträge pro Jahr
plt.subplot(2, 2, 3)
yearly_counts = combined_df.groupby('Jahr').size()
plt.bar(yearly_counts.index, yearly_counts.values, alpha=0.7)
plt.title('Anzahl Immobilienangebote pro Jahr')
plt.xlabel('Jahr')
plt.ylabel('Anzahl Angebote')
for year, count in yearly_counts.items():
    plt.annotate(f'{count}', (year, count), textcoords="offset points", xytext=(0,5), ha='center')

# Subplot 4: Preisentwicklung nach Datenquelle
plt.subplot(2, 2, 4)
for source in combined_df['Dataset'].unique():
    source_data = combined_df[combined_df['Dataset'] == source]
    yearly_source_avg = source_data.groupby('Jahr')['price'].mean()
    plt.plot(yearly_source_avg.index, yearly_source_avg.values, marker='o', label=source, linewidth=2)

plt.title('Preisentwicklung nach Datenquelle')
plt.xlabel('Jahr')
plt.ylabel('Durchschnittspreis (€)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Preissteigerungsraten berechnen
print("\nPreissteigerungsraten:")
years = sorted(combined_df['Jahr'].unique())
for i in range(1, len(years)):
    prev_year = years[i-1]
    curr_year = years[i]
    
    prev_price = combined_df[combined_df['Jahr'] == prev_year]['price'].mean()
    curr_price = combined_df[combined_df['Jahr'] == curr_year]['price'].mean()
    
    change_pct = ((curr_price - prev_price) / prev_price) * 100
    print(f"{prev_year} → {curr_year}: {change_pct:+.1f}% ({prev_price:.0f}€ → {curr_price:.0f}€)")

### 3.2 Bezirksvergleiche und geografische Analyse
Analysiere die Mietpreise nach Berliner Bezirken und identifiziere Trends.

In [ ]:
# Bezirksvergleiche und geografische Analyse
print("=" * 60)
print("BEZIRKSVERGLEICHE UND GEOGRAFISCHE ANALYSE")
print("=" * 60)

# Statistik pro Bezirk
print("\nMietpreisstatistik nach Bezirken:")
district_stats = combined_df.groupby('Bezirk')['price'].agg(['count', 'mean', 'median', 'std', 'min', 'max']).round(2)
district_stats = district_stats.sort_values('mean', ascending=False)
print(district_stats)

# Top 10 teuerste und günstigste Bezirke
print("\nTop 10 teuerste Bezirke (Durchschnitt):")
top_expensive = district_stats.head(10)
for district, stats in top_expensive.iterrows():
    print(f"  {district}: {stats['mean']:.2f} € (Median: {stats['median']:.2f} €, n={stats['count']})")

print("\nTop 10 günstigste Bezirke (Durchschnitt):")
top_cheap = district_stats.tail(10)
for district, stats in top_cheap.iterrows():
    print(f"  {district}: {stats['mean']:.2f} € (Median: {stats['median']:.2f} €, n={stats['count']})")

# Visualisierung der Bezirkspreise
plt.figure(figsize=(20, 12))

# Subplot 1: Durchschnittspreis pro Bezirk (Balkendiagramm)
plt.subplot(2, 2, 1)
district_avg = combined_df.groupby('Bezirk')['price'].mean().sort_values(ascending=False)
bars = plt.bar(range(len(district_avg)), district_avg.values, alpha=0.7)
plt.title('Durchschnittliche Mietpreise pro Bezirk', fontsize=14, fontweight='bold')
plt.xlabel('Bezirk')
plt.ylabel('Durchschnittspreis (€)')
plt.xticks(range(len(district_avg)), district_avg.index, rotation=45, ha='right')
plt.grid(True, alpha=0.3)

# Farbkodierung nach Preis
colors = plt.cm.RdYlBu_r(np.linspace(0, 1, len(bars)))
for bar, color in zip(bars, colors):
    bar.set_color(color)

# Subplot 2: Boxplot der Preisverteilung pro Bezirk
plt.subplot(2, 2, 2)
combined_df.boxplot(column='price', by='Bezirk', ax=plt.gca(), rot=45)
plt.title('Preisverteilung pro Bezirk')
plt.xlabel('Bezirk')
plt.ylabel('Preis (€)')
plt.suptitle('')  # Entfernt den automatischen Titel

# Subplot 3: Anzahl Angebote pro Bezirk
plt.subplot(2, 2, 3)
district_counts = combined_df.groupby('Bezirk').size().sort_values(ascending=False)
plt.bar(range(len(district_counts)), district_counts.values, alpha=0.7)
plt.title('Anzahl Immobilienangebote pro Bezirk')
plt.xlabel('Bezirk')
plt.ylabel('Anzahl Angebote')
plt.xticks(range(len(district_counts)), district_counts.index, rotation=45, ha='right')

# Subplot 4: Preisentwicklung der Top 6 Bezirke über die Jahre
plt.subplot(2, 2, 4)
top_6_districts = district_avg.head(6).index

for district in top_6_districts:
    district_data = combined_df[combined_df['Bezirk'] == district]
    yearly_district_avg = district_data.groupby('Jahr')['price'].mean()
    plt.plot(yearly_district_avg.index, yearly_district_avg.values, marker='o', label=district, linewidth=2)

plt.title('Preisentwicklung Top 6 Bezirke')
plt.xlabel('Jahr')
plt.ylabel('Durchschnittspreis (€)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistische Signifikanz zwischen Bezirken testen
print("\nStatistische Signifikanz zwischen teuersten und günstigsten Bezirken:")
expensive_district = district_avg.index[0]
cheap_district = district_avg.index[-1]

expensive_prices = combined_df[combined_df['Bezirk'] == expensive_district]['price']
cheap_prices = combined_df[combined_df['Bezirk'] == cheap_district]['price']

t_stat, p_value = stats.ttest_ind(expensive_prices, cheap_prices)
print(f"T-Test zwischen {expensive_district} und {cheap_district}:")
print(f"  T-Statistik: {t_stat:.3f}")
print(f"  P-Wert: {p_value:.6f}")
print(f"  Signifikant (p < 0.05): {'Ja' if p_value < 0.05 else 'Nein'}")

# Korrelationsanalyse
print("\nKorrelationsanalyse numerischer Variablen:")
numeric_columns = combined_df.select_dtypes(include=[np.number]).columns
if len(numeric_columns) > 1:
    correlation_matrix = combined_df[numeric_columns].corr()
    print(correlation_matrix.round(3))
    
    # Heatmap der Korrelationen
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
                square=True, fmt='.2f', cbar_kws={'label': 'Korrelation'})
    plt.title('Korrelationsmatrix numerischer Variablen')
    plt.tight_layout()
    plt.show()
else:
    print("Nicht genügend numerische Variablen für Korrelationsanalyse")

### 3.3 Wohnungsgrößen und Preis-pro-Quadratmeter-Analyse
Analysiere die Beziehung zwischen Wohnungsgröße und Mietpreis.

In [ ]:
# Wohnungsgrößen und Preis-pro-Quadratmeter-Analyse
print("=" * 60)
print("WOHNUNGSGRÖSSENS- UND PREIS-PRO-QUADRATMETER-ANALYSE")
print("=" * 60)

# Prüfe welche Spalten für Größenanalyse verfügbar sind
print("Verfügbare Spalten für Größenanalyse:")
size_columns = [col for col in combined_df.columns if any(keyword in col.lower() for keyword in ['size', 'größe', 'sqm', 'm2', 'area', 'fläche'])]
print(f"Größen-relevante Spalten: {size_columns}")

# Prüfe ob Preis-pro-Quadratmeter bereits berechnet wurde
price_per_sqm_columns = [col for col in combined_df.columns if any(keyword in col.lower() for keyword in ['price_per_sqm', 'preis_pro_m2', 'qm_preis'])]
print(f"Preis-pro-Quadratmeter Spalten: {price_per_sqm_columns}")

# Analysiere verfügbare Größendaten
if size_columns:
    for col in size_columns:
        print(f"\nAnalyse von {col}:")
        non_null_count = combined_df[col].notna().sum()
        print(f"  Verfügbare Werte: {non_null_count}/{len(combined_df)} ({non_null_count/len(combined_df)*100:.1f}%)")
        
        if non_null_count > 0:
            print(f"  Statistik: Min={combined_df[col].min():.1f}, Max={combined_df[col].max():.1f}, Median={combined_df[col].median():.1f}")
            
            # Visualisierung der Größenverteilung
            plt.figure(figsize=(12, 4))
            
            plt.subplot(1, 2, 1)
            plt.hist(combined_df[col].dropna(), bins=30, alpha=0.7, edgecolor='black')
            plt.title(f'Verteilung von {col}')
            plt.xlabel(col)
            plt.ylabel('Häufigkeit')
            
            plt.subplot(1, 2, 2)
            plt.boxplot(combined_df[col].dropna())
            plt.title(f'Boxplot von {col}')
            plt.ylabel(col)
            
            plt.tight_layout()
            plt.show()

# Preis-pro-Quadratmeter-Analyse falls verfügbar
if price_per_sqm_columns:
    for col in price_per_sqm_columns:
        print(f"\nAnalyse von {col}:")
        non_null_count = combined_df[col].notna().sum()
        print(f"  Verfügbare Werte: {non_null_count}/{len(combined_df)} ({non_null_count/len(combined_df)*100:.1f}%)")
        
        if non_null_count > 0:
            print(f"  Statistik: Min={combined_df[col].min():.2f}€, Max={combined_df[col].max():.2f}€, Median={combined_df[col].median():.2f}€")
            
            # Preis-pro-Quadratmeter nach Bezirk
            print(f"\nDurchschnittlicher {col} nach Bezirk:")
            price_per_sqm_by_district = combined_df.groupby('Bezirk')[col].mean().sort_values(ascending=False)
            for district, price in price_per_sqm_by_district.head(10).items():
                print(f"  {district}: {price:.2f} €/m²")
            
            # Visualisierung
            plt.figure(figsize=(15, 10))
            
            # Subplot 1: Preis-pro-Quadratmeter Verteilung
            plt.subplot(2, 2, 1)
            plt.hist(combined_df[col].dropna(), bins=30, alpha=0.7, edgecolor='black')
            plt.title(f'Verteilung von {col}')
            plt.xlabel(f'{col} (€/m²)')
            plt.ylabel('Häufigkeit')
            
            # Subplot 2: Preis-pro-Quadratmeter nach Bezirk
            plt.subplot(2, 2, 2)
            district_prices = combined_df.groupby('Bezirk')[col].mean().sort_values(ascending=False)
            plt.bar(range(len(district_prices)), district_prices.values, alpha=0.7)
            plt.title(f'Durchschnittlicher {col} nach Bezirk')
            plt.xlabel('Bezirk')
            plt.ylabel(f'{col} (€/m²)')
            plt.xticks(range(len(district_prices)), district_prices.index, rotation=45, ha='right')
            
            # Subplot 3: Preis-pro-Quadratmeter über die Jahre
            plt.subplot(2, 2, 3)
            yearly_price_per_sqm = combined_df.groupby('Year')[col].mean()
            plt.plot(yearly_price_per_sqm.index, yearly_price_per_sqm.values, marker='o', linewidth=2)
            plt.title(f'Entwicklung von {col} über die Jahre')
            plt.xlabel('Jahr')
            plt.ylabel(f'{col} (€/m²)')
            plt.grid(True, alpha=0.3)
            
            # Subplot 4: Boxplot nach Jahren
            plt.subplot(2, 2, 4)
            combined_df.boxplot(column=col, by='Year', ax=plt.gca())
            plt.title(f'{col} Verteilung nach Jahren')
            plt.xlabel('Jahr')
            plt.ylabel(f'{col} (€/m²)')
            plt.suptitle('')
            
            plt.tight_layout()
            plt.show()

# Korrelation zwischen Größe und Preis analysieren
if size_columns and 'price' in combined_df.columns:
    print("\nKorrelation zwischen Wohnungsgröße und Gesamtpreis:")
    
    for size_col in size_columns:
        # Nur Zeilen mit beiden Werten
        subset = combined_df[[size_col, 'price']].dropna()
        
        if len(subset) > 10:  # Mindestens 10 Datenpunkte
            correlation = subset[size_col].corr(subset['price'])
            print(f"  {size_col} vs. price: {correlation:.3f}")
            
            # Streudiagramm
            plt.figure(figsize=(10, 6))
            plt.scatter(subset[size_col], subset['price'], alpha=0.6)
            plt.xlabel(f'{size_col}')
            plt.ylabel('Price (€)')
            plt.title(f'Korrelation: {size_col} vs. Preis (r={correlation:.3f})')
            
            # Trendlinie hinzufügen
            z = np.polyfit(subset[size_col], subset['price'], 1)
            p = np.poly1d(z)
            plt.plot(subset[size_col], p(subset[size_col]), "r--", alpha=0.8)
            
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()

# Zusammenfassung der Größenanalyse
print("\n" + "="*60)
print("ZUSAMMENFASSUNG DER GRÖSSENANALYSE")
print("="*60)

if size_columns:
    print(f"Verfügbare Größeninformationen: {len(size_columns)} Spalten")
    for col in size_columns:
        coverage = combined_df[col].notna().sum() / len(combined_df) * 100
        print(f"  {col}: {coverage:.1f}% Abdeckung")
else:
    print("Keine Größeninformationen verfügbar")

if price_per_sqm_columns:
    print(f"Verfügbare Preis-pro-Quadratmeter-Informationen: {len(price_per_sqm_columns)} Spalten")
    for col in price_per_sqm_columns:
        coverage = combined_df[col].notna().sum() / len(combined_df) * 100
        print(f"  {col}: {coverage:.1f}% Abdeckung")
else:
    print("Keine Preis-pro-Quadratmeter-Informationen verfügbar")